In [1]:
import tsai.all
import datetime
import pandas as pd
import torch
import numpy as np
import wandb
from fastai.callback.wandb import WandbCallback
from fastai.callback.tracker import EarlyStoppingCallback, SaveModelCallback
import time
import math

In [2]:
%run ../../shared_functions.py
%run ../../my_shared_functions.py

<Figure size 640x480 with 0 Axes>

In [3]:
DIR_INPUT = '../../../fraud-detection-handbook/simulated-data-transformed/data/'

BEGIN_DATE = "2018-06-11"
END_DATE = "2018-09-14"

print("Load  files")
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),transactions_df.TX_FRAUD.sum()))

output_feature="TX_FRAUD"

input_features=['TX_AMOUNT','TX_DURING_WEEKEND', 'TX_DURING_NIGHT', 'CUSTOMER_ID_NB_TX_1DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW', 'CUSTOMER_ID_NB_TX_30DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW', 'TERMINAL_ID_NB_TX_1DAY_WINDOW',
       'TERMINAL_ID_RISK_1DAY_WINDOW', 'TERMINAL_ID_NB_TX_7DAY_WINDOW',
       'TERMINAL_ID_RISK_7DAY_WINDOW', 'TERMINAL_ID_NB_TX_30DAY_WINDOW',
       'TERMINAL_ID_RISK_30DAY_WINDOW']

BEGIN_DATE = "2018-08-08"
start_date_training = datetime.datetime.strptime(BEGIN_DATE, "%Y-%m-%d")
delta_train=7
delta_delay=7
delta_test=7

delta_valid = delta_test

(train_df, valid_df)=get_train_test_set(transactions_df,start_date_training,
                                       delta_train=delta_train,delta_delay=delta_delay,delta_test=delta_test)

SEQ_LEN = 5
SEED = 42

if torch.cuda.is_available():
    DEVICE = "cuda" 
else:
    DEVICE = "cpu"
print("Selected device is",DEVICE)

seed_everything(SEED)

(train_df, valid_df)=scaleData(train_df, valid_df,input_features)
x_train, y_train = prepare_sequenced_X_y(train_df, SEQ_LEN, input_features, output_feature)
x_valid, y_valid = prepare_sequenced_X_y(valid_df, SEQ_LEN, input_features, output_feature)
X, y, splits = tsai.all.combine_split_data([x_train.numpy(), x_valid.numpy()], [y_train.numpy(), y_valid.numpy()])

Load  files
CPU times: total: 562 ms
Wall time: 5.03 s
919767 transactions loaded, containing 8195 fraudulent transactions
Selected device is cuda


### FCN

In [6]:
config = dict(
 dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = SEED,
    begin_date = BEGIN_DATE,
    delta_train = delta_train,
    delta_delay = delta_delay,
    delta_test = delta_test,
    batch_size=64,
    num_workers=0,
    layers=[128, 256, 128],
    kss=[7, 5 ,3],
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=100,
    scale=True,
    criterion='bce',
    one_cycle_policy=False
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config, tags=['fcn', 'tsai', 'imbalance-not-considered', 'constant_lr_0_0001'])
config = wandb.config

In [7]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config.batch_size, num_workers=config.num_workers, drop_last=False, device=DEVICE)
model = tsai.all.FCN(
    c_in=dls.vars, 
    c_out=dls.c,
    layers=config.layers,
    kss=config.kss
    )

criterion = tsai.all.BCEWithLogitsLossFlat()
learn = tsai.all.ts_learner(
    dls,
    model,
    loss_func=criterion,
    opt_func=tsai.all.Adam,
    device=DEVICE,
    metrics=[],
    model_dir='FCN',
    cbs=[
        WandbCallback(
            log='all',
            log_model=True,
            seed=config.seed),
        EarlyStoppingCallback(
            patience=config.early_stopping_patience
        ),
        SaveModelCallback(
            fname='fcn_lr_0_0001'
        )])

lr = 0.0001
wandb.config.update({"lr": lr})

start_time=time.time()
learn.fit(config.max_epochs)
training_execution_time=time.time()-start_time

start_time=time.time()
valid_predictions_logits = get_predictions_sequential(learn.model.cuda(), dls.valid)
prediction_execution_time=time.time()-start_time
predictions_df = valid_df
predictions_df['predictions'] = torch.nn.Sigmoid()(torch.FloatTensor(valid_predictions_logits)).numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])

wandb.log({'Training execution time': training_execution_time})
wandb.log({'Prediction execution time': prediction_execution_time})
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})
wandb.finish()

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.024034,0.020749,00:20
1,0.023505,0.020098,00:16
2,0.015954,0.020814,00:14
3,0.015790,0.021149,00:14


Better model found at epoch 0 with valid_loss value: 0.020749390125274658.
Better model found at epoch 1 with valid_loss value: 0.020098144188523293.
No improvement since epoch 1: early stopping


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
raw_loss,▄▂▁▂▁▁▄▁▁▂▆▁▁▂▁▁▅█▂█▁▁▁▁▆▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁


### gMLP

In [10]:
config = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = SEED,
    begin_date = BEGIN_DATE,
    delta_train = delta_train,
    delta_delay = delta_delay,
    delta_test = delta_test,
    batch_size=64,
    num_workers=0,
    seq_len=SEQ_LEN,
    patch_size=1,
    d_model=256,
    d_ffn=512,
    depth=6,
    optimizer='adam',
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=100,
    scale=True,
    criterion='bce',
    one_cycle_policy=False
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config, tags=['gMLP', 'tsai', 'imbalance-not-considered', 'constant_lr_0_0001'])
config = wandb.config

In [11]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config.batch_size, num_workers=config.num_workers, drop_last=False, device=DEVICE)
model = tsai.all.gMLP(
    dls.vars,
    dls.c,
    SEQ_LEN,
    patch_size=config.patch_size,
    d_model=config.d_model,
    d_ffn=config.d_ffn,
    depth=config.depth
    )

criterion = tsai.all.BCEWithLogitsLossFlat()
learn = tsai.all.ts_learner(
    dls,
    model,
    loss_func=criterion,
    opt_func=tsai.all.Adam,
    device=DEVICE,
    metrics=[],
    model_dir='gMLP',
    cbs=[
        WandbCallback(
            log='all',
            log_model=True,
            seed=config.seed),
        EarlyStoppingCallback(
            patience=config.early_stopping_patience
        ),
        SaveModelCallback(
            fname='gmlp_lr_0_0001'
        )])

lr = 0.0001
wandb.config.update({"lr": lr})

start_time=time.time()
learn.fit(config.max_epochs)
training_execution_time=time.time()-start_time

start_time=time.time()
valid_predictions_logits = get_predictions_sequential(learn.model.cuda(), dls.valid)
prediction_execution_time=time.time()-start_time
predictions_df = valid_df
predictions_df['predictions'] = torch.nn.Sigmoid()(torch.FloatTensor(valid_predictions_logits)).numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])

wandb.log({'Training execution time': training_execution_time})
wandb.log({'Prediction execution time': prediction_execution_time})
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})
wandb.finish()

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.025378,0.023089,00:40
1,0.020178,0.022079,00:41
2,0.020571,0.021778,00:41
3,0.027282,0.019753,00:42
4,0.015174,0.018535,00:42
5,0.036517,0.018138,00:42
6,0.020246,0.018305,00:42
7,0.020029,0.018397,00:42


Better model found at epoch 0 with valid_loss value: 0.023089470341801643.
Better model found at epoch 1 with valid_loss value: 0.02207864634692669.
Better model found at epoch 2 with valid_loss value: 0.0217781700193882.
Better model found at epoch 3 with valid_loss value: 0.019752606749534607.
Better model found at epoch 4 with valid_loss value: 0.0185353085398674.
Better model found at epoch 5 with valid_loss value: 0.018137997016310692.
No improvement since epoch 5: early stopping


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
raw_loss,▃▁▂▂▆▁▁▃▃▃▁▂▁▁▁▁▃▂▁▁▁█▁▁▁▁▁▁▁▁▁▁█▁▃▅▁▅▁▁


In [7]:
wandb.finish()

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇██
mom_0,███████████▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▂▂▂▁▁
raw_loss,█▇▆▅▄▃▃▂▃▂▂▂▁▁▂▁▁▁▁▂▁▁▁▂▂▁▂▁▁▂▂▁▁▁▁▁▁▂▁▂
sqr_mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▇▆▆▅▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_samples_per_sec,▃▆▇▇▁▅▇▁▇▄▇▆▇▆▅▇█▇▇▇▇▅▄▆█▆▄▆▆█▆▇▆▅▄▆▆▄▆▄
wd_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,0.44995
eps_0,1e-05


### GRU-FCN

In [12]:
config = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = SEED,
    begin_date = BEGIN_DATE,
    delta_train = delta_train,
    delta_delay = delta_delay,
    delta_test = delta_test,
    batch_size=64,
    num_workers=0,
    seq_len=SEQ_LEN,
    hidden_size = 100,
    rnn_layers = 1,
    bias=True,
    rnn_dropout=0.8,
    cell_dropout=0,
    conv_layers=[128, 256, 128],
    kss=[7, 5, 3],
    bidirectional=False,
    fc_dropout=0,
    se=0,
    optimizer='adam',
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=100,
    scale=True,
    criterion='bce',
    one_cycle_policy=False
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config, tags=['gru-fcn', 'tsai', 'imbalance-not-considered', 'constant_lr_0_0001'])
config = wandb.config

In [13]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config.batch_size, num_workers=config.num_workers, drop_last=False, device=DEVICE)
model = tsai.all.GRU_FCN(
    c_in=dls.vars, 
    c_out=dls.c,
    seq_len=SEQ_LEN,
    hidden_size=config.hidden_size,
    rnn_layers=config.rnn_layers,
    bias=config.bias,
    cell_dropout=config.cell_dropout,
    rnn_dropout=config.rnn_dropout,
    bidirectional=config.bidirectional,
    fc_dropout=config.fc_dropout,
    conv_layers=config.conv_layers,
    kss=config.kss,
    se=config.se
    )

criterion = tsai.all.BCEWithLogitsLossFlat()
learn = tsai.all.ts_learner(
    dls,
    model,
    loss_func=criterion,
    opt_func=tsai.all.Adam,
    device=DEVICE,
    metrics=[],
    model_dir='GRU_FCN',
    cbs=[
        WandbCallback(
            log='all',
            log_model=True,
            seed=config.seed),
        EarlyStoppingCallback(
            patience=config.early_stopping_patience
        ),
        SaveModelCallback(
            fname='gru_fcn_lr_0_0001'
        )])

lr = 0.0001
wandb.config.update({"lr": lr})

start_time=time.time()
learn.fit(config.max_epochs)
training_execution_time=time.time()-start_time

start_time=time.time()
valid_predictions_logits = get_predictions_sequential(learn.model.cuda(), dls.valid)
prediction_execution_time=time.time()-start_time
predictions_df = valid_df
predictions_df['predictions'] = torch.nn.Sigmoid()(torch.FloatTensor(valid_predictions_logits)).numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])

wandb.log({'Training execution time': training_execution_time})
wandb.log({'Prediction execution time': prediction_execution_time})
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})
wandb.finish()

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.025102,0.023852,00:19
1,0.023193,0.021895,00:18
2,0.018527,0.021124,00:19
3,0.016555,0.024213,00:17
4,0.018923,0.021209,00:17


Better model found at epoch 0 with valid_loss value: 0.023852398619055748.
Better model found at epoch 1 with valid_loss value: 0.02189500816166401.
Better model found at epoch 2 with valid_loss value: 0.02112363465130329.
No improvement since epoch 2: early stopping


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
raw_loss,▅▂▁▂▁█▁▁▂▁▄▁▁▁▁▁▁▁▁█▁▁▁▁▅█▂▆▁▇▂▁▁▁▁▅▁▁▁▁


### GRU

In [14]:
config = dict(
   dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = SEED,
    begin_date = BEGIN_DATE,
    delta_train = delta_train,
    delta_delay = delta_delay,
    delta_test = delta_test,
    batch_size=64,
    num_workers=0,
    seq_len=SEQ_LEN,
    hidden_size = 100,
    n_layers = 1,
    rnn_dropout=0,
    bidirectional=False,
    fc_dropout=0,
    optimizer='adam',
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=100,
    scale=True,
    criterion='bce',
    one_cycle_policy=False
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config, tags=['gru', 'tsai', 'imbalance-not-considered', 'constant_lr_0_0001'])
config = wandb.config

In [16]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config.batch_size, num_workers=config.num_workers, drop_last=False, device=DEVICE)
model = tsai.all.GRU(
                c_in=dls.vars,
                c_out=dls.c,
                hidden_size=config.hidden_size,
                n_layers=config.n_layers,
                rnn_dropout=config.rnn_dropout,
                fc_dropout=config.fc_dropout)

criterion = tsai.all.BCEWithLogitsLossFlat()
learn = tsai.all.ts_learner(
    dls,
    model,
    loss_func=criterion,
    opt_func=tsai.all.Adam,
    device=DEVICE,
    metrics=[],
    model_dir='GRU',
    cbs=[
        WandbCallback(
            log='all',
            log_model=True,
            seed=config.seed),
        EarlyStoppingCallback(
            patience=config.early_stopping_patience
        ),
        SaveModelCallback(
            fname='gru_lr_0_0001'
        )])

lr = 0.0001
wandb.config.update({"lr": lr})

start_time=time.time()
learn.fit(config.max_epochs)
training_execution_time=time.time()-start_time

start_time=time.time()
valid_predictions_logits = get_predictions_sequential(learn.model.cuda(), dls.valid)
prediction_execution_time=time.time()-start_time
predictions_df = valid_df
predictions_df['predictions'] = torch.nn.Sigmoid()(torch.FloatTensor(valid_predictions_logits)).numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])

wandb.log({'Training execution time': training_execution_time})
wandb.log({'Prediction execution time': prediction_execution_time})
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})
wandb.finish()

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.026169,0.021906,00:11
1,0.021071,0.019581,00:10
2,0.016488,0.021001,00:11
3,0.019097,0.018579,00:10
4,0.015619,0.019624,00:10
5,0.019841,0.018508,00:10
6,0.021009,0.018318,00:12
7,0.015978,0.018364,00:11
8,0.011710,0.018960,00:12


Better model found at epoch 0 with valid_loss value: 0.021905837580561638.
Better model found at epoch 1 with valid_loss value: 0.019580887630581856.
Better model found at epoch 3 with valid_loss value: 0.018578602001070976.
Better model found at epoch 5 with valid_loss value: 0.018507586792111397.
Better model found at epoch 6 with valid_loss value: 0.01831795461475849.
No improvement since epoch 6: early stopping


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
raw_loss,▄▂▁▁▁▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁█▁▅▁▁▁▁▂▁▂▁


### InceptionTime

In [17]:
config = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = SEED,
    begin_date = BEGIN_DATE,
    delta_train = delta_train,
    delta_delay = delta_delay,
    delta_test = delta_test,
    batch_size=64,
    num_workers=0,
    seq_len=SEQ_LEN,
    nf=32,
    nb_filters=None,
    optimizer='adam',
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=100,
    scale=True,
    criterion='bce',
    one_cycle_policy=False
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config, tags=['inceptiontime', 'tsai', 'imbalance-not-considered', 'constant_lr_0_0001'])
config = wandb.config

In [18]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config.batch_size, num_workers=config.num_workers, drop_last=False, device=DEVICE)
model = tsai.all.InceptionTime(
    c_in=dls.vars,
    c_out=dls.c,
    nf=config.nf,
    nb_filters=config.nb_filters)

criterion = tsai.all.BCEWithLogitsLossFlat()
learn = tsai.all.ts_learner(
    dls,
    model,
    loss_func=criterion,
    opt_func=tsai.all.Adam,
    device=DEVICE,
    metrics=[],
    model_dir='InceptionTime',
    cbs=[
        WandbCallback(
            log='all',
            log_model=True,
            seed=config.seed),
        EarlyStoppingCallback(
            patience=config.early_stopping_patience
        ),
        SaveModelCallback(
            fname='inceptiontime_lr_0_0001'
        )])

lr = 0.0001
wandb.config.update({"lr": lr})

start_time=time.time()
learn.fit(config.max_epochs)
training_execution_time=time.time()-start_time

start_time=time.time()
valid_predictions_logits = get_predictions_sequential(learn.model.cuda(), dls.valid)
prediction_execution_time=time.time()-start_time
predictions_df = valid_df
predictions_df['predictions'] = torch.nn.Sigmoid()(torch.FloatTensor(valid_predictions_logits)).numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])

wandb.log({'Training execution time': training_execution_time})
wandb.log({'Prediction execution time': prediction_execution_time})
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})
wandb.finish()

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.023184,0.025800,00:48
1,0.021659,0.021903,00:48
2,0.018549,0.020355,00:50
3,0.011551,0.023795,00:50
4,0.019922,0.022235,00:50


Better model found at epoch 0 with valid_loss value: 0.02580038271844387.
Better model found at epoch 1 with valid_loss value: 0.02190282568335533.
Better model found at epoch 2 with valid_loss value: 0.020355477929115295.
No improvement since epoch 2: early stopping


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
raw_loss,▂▁▁▁▁▁▁▁▁▁▁▅▁▁▁▆▂▃▁▇▄▆█▆▁▃▄▃▁▁▅▅▁▁▁▁▂▂▁▁


### LSTM-FCN

In [19]:
config = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = SEED,
    begin_date = BEGIN_DATE,
    delta_train = delta_train,
    delta_delay = delta_delay,
    delta_test = delta_test,
    batch_size=64,
    num_workers=0,
    seq_len=SEQ_LEN,
    hidden_size = 100,
    rnn_layers = 1,
    bias=True,
    rnn_dropout=0.8,
    cell_dropout=0,
    conv_layers=[128, 256, 128],
    kss=[7, 5, 3],
    bidirectional=False,
    fc_dropout=0,
    se=0, # squeeze-and-excite, which equals to 16 for MLSTM-FCN
    optimizer='adam',
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=100,
    scale=True,
    criterion='bce',
    one_cycle_policy=False
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config, tags=['lstm-fcn', 'tsai', 'imbalance-not-considered', 'constant_lr_0_0001'])
config = wandb.config

In [20]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config.batch_size, num_workers=config.num_workers, drop_last=False, device=DEVICE)
model = tsai.all.LSTM_FCN(
                c_in=dls.vars, 
                c_out=dls.c,
                seq_len=SEQ_LEN,
                hidden_size=config.hidden_size,
                rnn_layers=config.rnn_layers,
                bias=config.bias,
                cell_dropout=config.cell_dropout,
                rnn_dropout=config.rnn_dropout,
                bidirectional=config.bidirectional,
                fc_dropout=config.fc_dropout,
                conv_layers=config.conv_layers,
                kss=config.kss,
                se=config.se
                )

criterion = tsai.all.BCEWithLogitsLossFlat()
learn = tsai.all.ts_learner(
    dls,
    model,
    loss_func=criterion,
    opt_func=tsai.all.Adam,
    device=DEVICE,
    metrics=[],
    model_dir='LSTM_FCN',
    cbs=[
        WandbCallback(
            log='all',
            log_model=True,
            seed=config.seed),
        EarlyStoppingCallback(
            patience=config.early_stopping_patience
        ),
        SaveModelCallback(
            fname='lstm_fcn_lr_0_0001'
        )])

lr = 0.0001
wandb.config.update({"lr": lr})

start_time=time.time()
learn.fit(config.max_epochs)
training_execution_time=time.time()-start_time

start_time=time.time()
valid_predictions_logits = get_predictions_sequential(learn.model.cuda(), dls.valid)
prediction_execution_time=time.time()-start_time
predictions_df = valid_df
predictions_df['predictions'] = torch.nn.Sigmoid()(torch.FloatTensor(valid_predictions_logits)).numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])

wandb.log({'Training execution time': training_execution_time})
wandb.log({'Prediction execution time': prediction_execution_time})
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})
wandb.finish()

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.027611,0.022119,00:19
1,0.022616,0.020538,00:19
2,0.017323,0.021090,00:18
3,0.017336,0.021698,00:20


Better model found at epoch 0 with valid_loss value: 0.022119425237178802.
Better model found at epoch 1 with valid_loss value: 0.02053755708038807.
No improvement since epoch 1: early stopping


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
raw_loss,▁▄▃▃▁▁▂▁▄▁▂▄▁▂▁▄▅▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁


### LSTM

In [21]:
config = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = SEED,
    begin_date = BEGIN_DATE,
    delta_train = delta_train,
    delta_delay = delta_delay,
    delta_test = delta_test,
    batch_size=64,
    num_workers=0,
    seq_len=SEQ_LEN,
    hidden_size = 100,
    n_layers = 1,
    rnn_dropout=0,
    bidirectional=False,
    fc_dropout=0,
    optimizer='adam',
    lr=0.0001,
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=100,
    scale=True,
    criterion='bce',
    one_cycle_policy=False
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config, tags=['lstm', 'tsai', 'imbalance-not-considered', 'constant_lr_0_0001'])
config = wandb.config

In [45]:
model = tsai.all.LSTM(
                c_in=dls.vars,
                c_out=dls.c)
model

LSTM(
  (rnn): LSTM(15, 100, batch_first=True)
  (dropout): Identity()
  (fc): Linear(in_features=100, out_features=1, bias=True)
)

In [46]:
from tsai.models.utils import count_parameters
count_parameters(model)

46901

In [22]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config.batch_size, num_workers=config.num_workers, drop_last=False, device=DEVICE)
model = tsai.all.LSTM(
                c_in=dls.vars,
                c_out=dls.c,
                hidden_size=config.hidden_size,
                n_layers=config.n_layers,
                rnn_dropout=config.rnn_dropout,
                fc_dropout=config.fc_dropout)

criterion = tsai.all.BCEWithLogitsLossFlat()
learn = tsai.all.ts_learner(
    dls,
    model,
    loss_func=criterion,
    opt_func=tsai.all.Adam,
    device=DEVICE,
    metrics=[],
    model_dir='LSTM',
    cbs=[
        WandbCallback(
            log='all',
            log_model=True,
            seed=config.seed),
        EarlyStoppingCallback(
            patience=config.early_stopping_patience
        ),
        SaveModelCallback(
            fname='lstm_lr_0_0001'
        )])

lr = 0.0001
wandb.config.update({"lr": lr})

start_time=time.time()
learn.fit(config.max_epochs)
training_execution_time=time.time()-start_time

start_time=time.time()
valid_predictions_logits = get_predictions_sequential(learn.model.cuda(), dls.valid)
prediction_execution_time=time.time()-start_time
predictions_df = valid_df
predictions_df['predictions'] = torch.nn.Sigmoid()(torch.FloatTensor(valid_predictions_logits)).numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])

wandb.log({'Training execution time': training_execution_time})
wandb.log({'Prediction execution time': prediction_execution_time})
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})
wandb.finish()

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.021546,0.023072,00:11
1,0.027005,0.020980,00:12
2,0.027654,0.021764,00:11
3,0.026498,0.021364,00:11


Better model found at epoch 0 with valid_loss value: 0.023071594536304474.
Better model found at epoch 1 with valid_loss value: 0.020980414003133774.
No improvement since epoch 1: early stopping


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
raw_loss,▁▁▂▁▁▁▁▁▁▂▁▂▁▁▂▁▂▂▁▁█▁▁▁▁█▁▁▁▂▁▁▁▁▁▂▁▁▁▁


### MLSTM-FCN

In [23]:
config = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = SEED,
    begin_date = BEGIN_DATE,
    delta_train = delta_train,
    delta_delay = delta_delay,
    delta_test = delta_test,
    batch_size=64,
    num_workers=0,
    seq_len=SEQ_LEN,
    hidden_size = 100,
    rnn_layers = 1,
    bias=True,
    rnn_dropout=0.8,
    cell_dropout=0,
    conv_layers=[128, 256, 128],
    kss=[7, 5, 3],
    bidirectional=False,
    fc_dropout=0,
    se=16, # squeeze-and-excite
    optimizer='adam',
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=100,
    scale=True,
    criterion='bce',
    one_cycle_policy=False
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config, tags=['mlstm-fcn', 'tsai', 'imbalance-not-considered', 'constant_lr_0_0001'])
config = wandb.config

In [24]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config.batch_size, num_workers=config.num_workers, drop_last=False, device=DEVICE)
model = tsai.all.MLSTM_FCN(
    c_in=dls.vars, 
    c_out=dls.c,
    seq_len=SEQ_LEN,
    hidden_size=config.hidden_size,
    rnn_layers=config.rnn_layers,
    bias=config.bias,
    cell_dropout=config.cell_dropout,
    rnn_dropout=config.rnn_dropout,
    bidirectional=config.bidirectional,
    fc_dropout=config.fc_dropout,
    conv_layers=config.conv_layers,
    kss=config.kss,
    se=config.se
    )

criterion = tsai.all.BCEWithLogitsLossFlat()
learn = tsai.all.ts_learner(
    dls,
    model,
    loss_func=criterion,
    opt_func=tsai.all.Adam,
    device=DEVICE,
    metrics=[],
    model_dir='MLSTM_LSTM',
    cbs=[
        WandbCallback(
            log='all',
            log_model=True,
            seed=config.seed),
        EarlyStoppingCallback(
            patience=config.early_stopping_patience
        ),
        SaveModelCallback(
            fname='mlstm_fcn_lr_0_0001'
        )])

lr = 0.0001
wandb.config.update({"lr": lr})

start_time=time.time()
learn.fit(config.max_epochs)
training_execution_time=time.time()-start_time

start_time=time.time()
valid_predictions_logits = get_predictions_sequential(learn.model.cuda(), dls.valid)
prediction_execution_time=time.time()-start_time
predictions_df = valid_df
predictions_df['predictions'] = torch.nn.Sigmoid()(torch.FloatTensor(valid_predictions_logits)).numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])

wandb.log({'Training execution time': training_execution_time})
wandb.log({'Prediction execution time': prediction_execution_time})
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})
wandb.finish()

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.025065,0.021484,00:25
1,0.028185,0.019887,00:24
2,0.018854,0.019748,00:23
3,0.024754,0.019842,00:23
4,0.012645,0.021126,00:23


Better model found at epoch 0 with valid_loss value: 0.02148369327187538.
Better model found at epoch 1 with valid_loss value: 0.019887438043951988.
Better model found at epoch 2 with valid_loss value: 0.019747572019696236.
No improvement since epoch 2: early stopping


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
raw_loss,▁▅▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▂▃▃▁


### ResCNN

In [25]:
config = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = SEED,
    begin_date = BEGIN_DATE,
    delta_train = delta_train,
    delta_delay = delta_delay,
    delta_test = delta_test,
    batch_size=64,
    num_workers=0,
    seq_len=SEQ_LEN,
    coord=False,
    separable=False,
    zero_norm=False,
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=100,
    scale=True,
    criterion='bce',
    one_cycle_policy=False
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config, tags=['rescnn', 'tsai', 'imbalance-not-considered', 'constant_lr_0_0001'])
config = wandb.config

In [26]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config.batch_size, num_workers=config.num_workers, drop_last=False, device=DEVICE)
model = tsai.all.ResCNN(
                c_in=dls.vars, 
                c_out=dls.c,
                coord=config.coord,
                separable=config.separable,
                zero_norm=config.zero_norm
            )

criterion = tsai.all.BCEWithLogitsLossFlat()
learn = tsai.all.ts_learner(
    dls,
    model,
    loss_func=criterion,
    opt_func=tsai.all.Adam,
    device=DEVICE,
    metrics=[],
    model_dir='ResCNN',
        cbs=[
        WandbCallback(
            log='all',
            log_model=True,
            seed=config.seed),
        EarlyStoppingCallback(
            patience=config.early_stopping_patience
        ),
        SaveModelCallback(
            fname='rescnn_lr_0_0001'
        )])

lr = 0.0001
wandb.config.update({"lr": lr})

start_time=time.time()
learn.fit(config.max_epochs)
training_execution_time=time.time()-start_time

start_time=time.time()
valid_predictions_logits = get_predictions_sequential(learn.model.cuda(), dls.valid)
prediction_execution_time=time.time()-start_time
predictions_df = valid_df
predictions_df['predictions'] = torch.nn.Sigmoid()(torch.FloatTensor(valid_predictions_logits)).numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])

wandb.log({'Training execution time': training_execution_time})
wandb.log({'Prediction execution time': prediction_execution_time})
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})
wandb.finish()

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.026221,0.020549,00:23
1,0.019287,0.020299,00:23
2,0.018683,0.019413,00:22
3,0.018220,0.019890,00:22
4,0.019829,0.020259,00:22


Better model found at epoch 0 with valid_loss value: 0.02054893597960472.
Better model found at epoch 1 with valid_loss value: 0.0202992744743824.
Better model found at epoch 2 with valid_loss value: 0.019412841647863388.
No improvement since epoch 2: early stopping


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
raw_loss,▅▂▁▆▁▁▁▁█▆▁▁▁▁▁▁▃▁▁▁▁▁▅▁▆▁▆▁▁▇▁▁▁▁▁▁▁▁▁▁


### TSiT

In [41]:
config = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = SEED,
    begin_date = BEGIN_DATE,
    delta_train = delta_train,
    delta_delay = delta_delay,
    delta_test = delta_test,
    batch_size=64,
    num_workers=0,
    seq_len=SEQ_LEN,
    d_model=128,
    depth=6,
    n_heads=16,
    act='gelu',
    lsa=False,
    attn_dropout=0,
    dropout=0,
    drop_path_rate=0,
    mlp_ratio=1,
    qkv_bias=True,
    pre_norm=False,
    use_token=False, # use_token set to False as c_out == 1
    use_pe=True,
    cat_pos=None,
    n_cat_embeds=None,
    cat_embed_dims=None,
    cat_padding_idxs=None,
    token_size=None,
    tokenizer=None,
    feature_extractor=None,
    flatten=False,
    concat_pool=True,
    fc_dropout=0,
    use_bn=False,
    bias_init=None,
    y_range=None,
    custom_head=None,
    optimizer='adam',
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=100,
    scale=True,
    criterion='bce',
    one_cycle_policy=False
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config, tags=['tsit', 'transformer', 'tsai', 'imbalance-not-considered', 'constant_lr_0_0001'])
config = wandb.config

In [42]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config.batch_size, num_workers=config.num_workers, drop_last=False, device=DEVICE)
model = tsai.all.TSiT(
    c_in=dls.vars,
    c_out=dls.c,
    seq_len=SEQ_LEN,
    d_model=config.d_model,
    depth=config.depth,
    n_heads=config.n_heads,
    act=config.act,
    lsa=config.lsa,
    attn_dropout=config.attn_dropout,
    dropout=config.dropout,
    drop_path_rate=config.drop_path_rate,
    mlp_ratio=config.mlp_ratio,
    qkv_bias=config.qkv_bias,
    pre_norm=config.pre_norm,
    use_token=config.use_token,
    use_pe=config.use_pe,
    cat_pos=config.cat_pos,
    n_cat_embeds=config.n_cat_embeds,
    cat_embed_dims=config.cat_embed_dims,
    cat_padding_idxs=config.cat_padding_idxs,
    token_size=config.token_size,
    tokenizer=config.tokenizer,
    feature_extractor=config.feature_extractor,
    flatten=config.flatten,
    concat_pool=config.concat_pool,
    fc_dropout=config.fc_dropout,
    use_bn=config.use_bn,
    bias_init=config.bias_init,
    y_range=config.y_range,
    custom_head=config.custom_head
)

criterion = tsai.all.BCEWithLogitsLossFlat()
learn = tsai.all.ts_learner(
    dls,
    model,
    loss_func=criterion,
    opt_func=tsai.all.Adam,
    device=DEVICE,
    metrics=[],
    model_dir='TSiT',
        cbs=[
        WandbCallback(
            log='all',
            log_model=True,
            seed=config.seed),
        EarlyStoppingCallback(
            patience=config.early_stopping_patience
        ),
        SaveModelCallback(
            fname='tsit_lr_0_0001'
        )])

lr = 0.0001
wandb.config.update({"lr": lr})

start_time=time.time()
learn.fit(config.max_epochs)
training_execution_time=time.time()-start_time

start_time=time.time()
valid_predictions_logits = get_predictions_sequential(learn.model.cuda(), dls.valid)
prediction_execution_time=time.time()-start_time
predictions_df = valid_df
predictions_df['predictions'] = torch.nn.Sigmoid()(torch.FloatTensor(valid_predictions_logits)).numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])

wandb.log({'Training execution time': training_execution_time})
wandb.log({'Prediction execution time': prediction_execution_time})
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})
wandb.finish()

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.041065,0.041305,01:00
1,0.045372,0.036684,01:02
2,0.045896,0.038886,01:04
3,0.048359,0.040910,01:04


Better model found at epoch 0 with valid_loss value: 0.04130484536290169.
Better model found at epoch 1 with valid_loss value: 0.03668364882469177.
No improvement since epoch 1: early stopping


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


### TST

In [29]:
config = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = SEED,
    begin_date = BEGIN_DATE,
    delta_train = delta_train,
    delta_delay = delta_delay,
    delta_test = delta_test,
    batch_size=64,
    num_workers=0,
    seq_len=SEQ_LEN,
    max_seq_len=None,
    n_layers=3,
    d_model=128,
    n_heads=16,
    d_k=None,
    d_v=None,
    d_ff=256,
    dropout=0.1,
    act='gelu',
    fc_dropout=0.,
    optimizer='adam',
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=100,
    scale=True,
    criterion='bce',
    one_cycle_policy=False
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config, tags=['tst', 'transformer', 'tsai', 'imbalance-not-considered', 'constant_lr_0_0001'])
config = wandb.config

In [30]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config.batch_size, num_workers=config.num_workers, drop_last=False, device=DEVICE)
model = tsai.all.TST(
    c_in=dls.vars, 
    c_out=dls.c,
    seq_len=SEQ_LEN,
    n_layers=config.n_layers,
    d_model=config.d_model,
    n_heads=config.n_heads,
    d_k=config.d_k,
    d_v=config.d_v,
    d_ff=config.d_ff,
    dropout=config.dropout,
    act=config.act,
    fc_dropout=config.fc_dropout
    )

criterion = tsai.all.BCEWithLogitsLossFlat()
learn = tsai.all.ts_learner(
    dls,
    model,
    loss_func=criterion,
    opt_func=tsai.all.Adam,
    device=DEVICE,
    metrics=[],
    model_dir='TST',
    cbs=[
        WandbCallback(
            log='all',
            log_model=True,
            seed=config.seed),
        EarlyStoppingCallback(
            patience=config.early_stopping_patience
        ),
        SaveModelCallback(
            fname='tst_lr_0_0001'
        )])

lr = 0.0001
wandb.config.update({"lr": lr})

start_time=time.time()
learn.fit(config.max_epochs)
training_execution_time=time.time()-start_time

start_time=time.time()
valid_predictions_logits = get_predictions_sequential(learn.model.cuda(), dls.valid)
prediction_execution_time=time.time()-start_time
predictions_df = valid_df
predictions_df['predictions'] = torch.nn.Sigmoid()(torch.FloatTensor(valid_predictions_logits)).numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])

wandb.log({'Training execution time': training_execution_time})
wandb.log({'Prediction execution time': prediction_execution_time})
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})
wandb.finish()

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.028960,0.024479,00:36
1,0.020052,0.023787,00:36
2,0.030817,0.023426,00:36
3,0.027757,0.024906,00:36
4,0.020667,0.022197,00:37
5,0.020727,0.021645,00:36
6,0.020130,0.021402,00:37
7,0.021384,0.022421,00:36
8,0.028250,0.021398,00:36
9,0.018279,0.022066,00:36


Better model found at epoch 0 with valid_loss value: 0.024479037150740623.
Better model found at epoch 1 with valid_loss value: 0.02378668449819088.
Better model found at epoch 2 with valid_loss value: 0.023425934836268425.
Better model found at epoch 4 with valid_loss value: 0.02219652384519577.
Better model found at epoch 5 with valid_loss value: 0.021645039319992065.
Better model found at epoch 6 with valid_loss value: 0.021401671692728996.
Better model found at epoch 8 with valid_loss value: 0.021397890523076057.
Better model found at epoch 10 with valid_loss value: 0.020465144887566566.
Better model found at epoch 11 with valid_loss value: 0.01992981694638729.
Better model found at epoch 13 with valid_loss value: 0.01969936117529869.
No improvement since epoch 13: early stopping


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
raw_loss,▂▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▁▂▁▄▁▁▄▁▁▁▁▁▁▁


### XCM

In [31]:
config = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = SEED,
    begin_date = BEGIN_DATE,
    delta_train = delta_train,
    delta_delay = delta_delay,
    delta_test = delta_test,
    batch_size=64,
    num_workers=0,
    window_perc=1,
    flatten=False,
    concat_pool=False,
    fc_dropout=0,
    bn=False,
    y_range=None,
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=100,
    scale=True,
    criterion='bce',
    one_cycle_policy=False
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config, tags=['xcm', 'tsai', 'imbalance-not-considered', 'constant_lr_0_0001'])
config = wandb.config

In [32]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config.batch_size, num_workers=config.num_workers, drop_last=False, device=DEVICE)
model = tsai.all.XCM(
    c_in=dls.vars, 
    c_out=dls.c,
    seq_len=SEQ_LEN,
    window_perc=config.window_perc, # window_size = int(round(seq_len * window_perc, 0))
                                    # ...Conv2d(1, nf, kernel_size=(1, window_size))...
    flatten=config.flatten,         # keep False if possible
    concat_pool=config.concat_pool, # True or False
    fc_dropout=config.fc_dropout,   # float range
    bn=config.bn,                   # True or False
    y_range=config.y_range          # None or tuple(low, high) where fastai.SigmoidRange(low, high)
    )

criterion = tsai.all.BCEWithLogitsLossFlat()
learn = tsai.all.ts_learner(
    dls,
    model,
    loss_func=criterion,
    opt_func=tsai.all.Adam,
    device=DEVICE,
    metrics=[],
    model_dir='XCM',
     cbs=[
        WandbCallback(
            log='all',
            log_model=True,
            seed=config.seed),
        EarlyStoppingCallback(
            patience=config.early_stopping_patience
        ),
        SaveModelCallback(
            fname='xcm_lr_0_0001'
        )])

lr = 0.0001
wandb.config.update({"lr": lr})

start_time=time.time()
learn.fit(config.max_epochs)
training_execution_time=time.time()-start_time

start_time=time.time()
valid_predictions_logits = get_predictions_sequential(learn.model.cuda(), dls.valid)
prediction_execution_time=time.time()-start_time
predictions_df = valid_df
predictions_df['predictions'] = torch.nn.Sigmoid()(torch.FloatTensor(valid_predictions_logits)).numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])

wandb.log({'Training execution time': training_execution_time})
wandb.log({'Prediction execution time': prediction_execution_time})
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})
wandb.finish()

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.022633,0.022123,00:19
1,0.016347,0.020109,00:18
2,0.020248,0.021250,00:18
3,0.019169,0.020304,00:18


Better model found at epoch 0 with valid_loss value: 0.02212292142212391.
Better model found at epoch 1 with valid_loss value: 0.020108861848711967.
No improvement since epoch 1: early stopping


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
raw_loss,▅▃▁▁▁█▁▁▂▁▁▁▁▁▁▁▁▂▁▁▅▁▃▁▃▂▁▁▁▁▄▁▁▇▁▂▁▅▁▁


### OmniScaleCNN

In [33]:
config = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = SEED,
    begin_date = BEGIN_DATE,
    delta_train = delta_train,
    delta_delay = delta_delay,
    delta_test = delta_test,
    batch_size=64,
    num_workers=0,
    few_shot=False,
    layers=[8*128, 5 * 128 * 256 + 2 * 256  *128],
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=100,
    scale=True,
    criterion='bce',
    one_cycle_policy=False
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config, tags=['omniscalecnn', 'tsai', 'imbalance-not-considered', 'constant_lr_0_0001'])
config = wandb.config

In [34]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config.batch_size, num_workers=config.num_workers, drop_last=False, device=DEVICE)
model = tsai.all.OmniScaleCNN(
    c_in=dls.vars, 
    c_out=dls.c,
    seq_len=SEQ_LEN,
    layers=config.layers,
    few_shot=config.few_shot
    )

criterion = tsai.all.BCEWithLogitsLossFlat()
learn = tsai.all.ts_learner(
    dls,
    model,
    loss_func=criterion,
    opt_func=tsai.all.Adam,
    device=DEVICE,
    metrics=[],
    model_dir='OmniScaleCNN',
    cbs=[
        WandbCallback(
            log='all',
            log_model=True,
            seed=config.seed),
        EarlyStoppingCallback(
            patience=config.early_stopping_patience
        ),
        SaveModelCallback(
            fname='omniscalecnn_lr_0_0001'
        )])

lr = 0.0001
wandb.config.update({"lr": lr})

start_time=time.time()
learn.fit(config.max_epochs)
training_execution_time=time.time()-start_time

start_time=time.time()
valid_predictions_logits = get_predictions_sequential(learn.model.cuda(), dls.valid)
prediction_execution_time=time.time()-start_time
predictions_df = valid_df
predictions_df['predictions'] = torch.nn.Sigmoid()(torch.FloatTensor(valid_predictions_logits)).numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])

wandb.log({'Training execution time': training_execution_time})
wandb.log({'Prediction execution time': prediction_execution_time})
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})
wandb.finish()

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.022417,0.022963,00:27
1,0.023621,0.026011,00:28
2,0.017101,0.022229,00:28
3,0.017068,0.020788,00:32
4,0.025214,0.022184,00:31
5,0.019788,0.022716,00:32


Better model found at epoch 0 with valid_loss value: 0.022963499650359154.
Better model found at epoch 2 with valid_loss value: 0.02222934179008007.
Better model found at epoch 3 with valid_loss value: 0.02078813873231411.
No improvement since epoch 3: early stopping


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
raw_loss,▁▅▂▁▆▁█▃▁▄▁▁▁▁▃▁▁▁▁▁▁▁▁▁▄▃▂▁▄▁▁▁▁▁▁▁▂▁▅▃


### ResNet

In [35]:
config = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = SEED,
    begin_date = BEGIN_DATE,
    delta_train = delta_train,
    delta_delay = delta_delay,
    delta_test = delta_test,
    batch_size=64,
    num_workers=0,
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=100,
    scale=True,
    criterion='bce',
    one_cycle_policy=False
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config, tags=['resnet', 'tsai', 'imbalance-not-considered', 'constant_lr_0_0001'])
config = wandb.config

In [37]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config.batch_size, num_workers=config.num_workers, drop_last=False, device=DEVICE)
model = tsai.all.ResNet(
    c_in=dls.vars, 
    c_out=dls.c)

criterion = tsai.all.BCEWithLogitsLossFlat()
learn = tsai.all.ts_learner(
    dls,
    model,
    loss_func=criterion,
    opt_func=tsai.all.Adam,
    device=DEVICE,
    metrics=[],
    model_dir='ResNet',
    cbs=[
        WandbCallback(
            log='all',
            log_model=True,
            seed=config.seed),
        EarlyStoppingCallback(
            patience=config.early_stopping_patience
        ),
        SaveModelCallback(
            fname='resnet_lr_0_0001'
        )])

lr = 0.0001
wandb.config.update({"lr": lr})

start_time=time.time()
learn.fit(config.max_epochs)
training_execution_time=time.time()-start_time

start_time=time.time()
valid_predictions_logits = get_predictions_sequential(learn.model.cuda(), dls.valid)
prediction_execution_time=time.time()-start_time
predictions_df = valid_df
predictions_df['predictions'] = torch.nn.Sigmoid()(torch.FloatTensor(valid_predictions_logits)).numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])

wandb.log({'Training execution time': training_execution_time})
wandb.log({'Prediction execution time': prediction_execution_time})
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})
wandb.finish()

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.022417,0.022323,00:30
1,0.022463,0.020665,00:31
2,0.016707,0.023481,00:31
3,0.019217,0.020856,00:32


Better model found at epoch 0 with valid_loss value: 0.022322973236441612.
Better model found at epoch 1 with valid_loss value: 0.020665233954787254.
No improvement since epoch 1: early stopping


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
raw_loss,▂▁▁▃▁▁▁▅▁▁▁▁▁▁▃▁▁▂▁▁▃▅▁▁▁▂▅▁▁▁▁▁▁█▂▁▁▁▁▁
